In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import json

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from utils import *
from model.data_loader import *
from model.unet_baseline import unet
from model.octave_unet import o_unet
from model.unet_ternaus import ternausNet16
from model.unet_ternaus_tweaked import ternausNet16_tweaked
from model.unet_pix2pix import unet_pix2pix

In [3]:
# Paths
train_pth = 'data/membrane/train'
test_pth = 'data/membrane/test'
results_pth = 'results/'
pretrained_pth = 'pretrained/'

# Experiment A – baseline

Train the original unet model on the full 512px dataset until overfitting.

In [5]:
# Full available image size
img_sz = (800, 600, 1)
batch_sz = 2

# Models
models = [
    ['unet_baseline', unet, dict(input_size=img_sz, transpose=True)]
]

# Data loaders
train_loader = loader(train_pth, input_generator_train, target_generator_train, 
                      batch_sz=batch_sz, img_sz=img_sz[:2])
test_loader = loader(test_pth, input_generator_test, target_generator_test, 
                     batch_sz=batch_sz, img_sz=img_sz[:2])

In [11]:
# Results csv saved with this filename
test_title = '512px_500steps_10epochs'

# Experiment configuration
training_params = dict(
    train_steps=500, 
    val_steps=100, 
    epochs=10, 
    iterations=1, 
    lr=1e-4
)

# Train models and record results
for model in models:
    print(f'\nTESTING MODEL: {model[0]}')
    save_pth = f'{pretrained_pth}{model[0]}_{test_title}.h5'
    results = test_model(model[1], train_loader, test_loader, **training_params, 
                         model_params=model[2], save_pth=save_pth)
    results_df = hists2df(results)
    results_df.to_csv(f'{results_pth}{model[0]}_{test_title}.csv')


TESTING MODEL: unet_baseline
Epoch 1/10
500/500 [==============================] - 259s 519ms/step - loss: 0.5889 - acc: 0.8872 - val_loss: 0.4561 - val_acc: 0.9299
Epoch 2/10
500/500 [==============================] - 257s 515ms/step - loss: 0.4870 - acc: 0.9201 - val_loss: 0.4700 - val_acc: 0.9347
Epoch 3/10
500/500 [==============================] - 257s 514ms/step - loss: 0.4551 - acc: 0.9270 - val_loss: 0.4470 - val_acc: 0.9349
Epoch 4/10
500/500 [==============================] - 257s 514ms/step - loss: 0.4327 - acc: 0.9318 - val_loss: 0.4386 - val_acc: 0.9376
Epoch 5/10
499/500 [============================>.] - ETA: 0s - loss: 0.4139 - acc: 0.9358
Epoch 00005: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
500/500 [==============================] - 255s 511ms/step - loss: 0.4139 - acc: 0.9358 - val_loss: 0.4374 - val_acc: 0.9367
Epoch 6/10
499/500 [============================>.] - ETA: 0s - loss: 0.4003 - acc: 0.9396
Epoch 00006: ReduceLROnPlateau reducing

Best Validation Accuracy: 0.9376

# Experiment B – Slow test
Train all models under test for 20 epochs of 250 training steps.

In [4]:
# Half available image size to reduce training time
img_sz = (256, 256, 1)
batch_sz = 2

# Models
models = [
    ['unet_baseline',           unet,                 dict(input_size=img_sz, transpose=True)],
    ['unet_baseline_upsampled', unet,                 dict(input_size=img_sz, transpose=False)],
    ['unet_ternaus_nopre',      ternausNet16_tweaked, dict(input_size=img_sz, dropout=False, batch_norm=False, pretrained=False)],
    ['unet_ternaus',            ternausNet16,         dict(input_size=img_sz)],
    ['unet_ternaus_drop',       ternausNet16_tweaked, dict(input_size=img_sz, dropout=True,  batch_norm=False, pretrained=True)],
    ['unet_ternaus_bn',         ternausNet16_tweaked, dict(input_size=img_sz, dropout=False, batch_norm=True,  pretrained=True)],
    ['unet_ternaus_dropbn',     ternausNet16_tweaked, dict(input_size=img_sz, dropout=True,  batch_norm=True,  pretrained=True)],
    ['unet_pix2pix',            unet_pix2pix, dict(input_size=img_sz)]
]

# Data loaders
train_loader = loader(train_pth, input_generator_train, target_generator_train, batch_sz=batch_sz, img_sz=img_sz[:2])
test_loader = loader(test_pth, input_generator_test, target_generator_test, batch_sz=batch_sz, img_sz=img_sz[:2])

In [5]:
# Results csv saved with this filename
test_title = '256px_250steps_20epochs'

# Experiment configuration
training_params = dict(
    train_steps=250, 
    val_steps=100, 
    epochs=20, 
    iterations=5, 
    lr=1e-4
)

# Train models and record results
for model in models:
    print(f'\nTESTING MODEL: {model[0]}')
    save_pth = f'{pretrained_pth}{model[0]}_{test_title}.h5'
    results = test_model(model[1], train_loader, test_loader, **training_params, 
                         model_params=model[2], save_pth=save_pth)
    results_df = hists2df(results)
    results_df.to_csv(f'{results_pth}{model[0]}_{test_title}.csv')


TESTING MODEL: unet_pix2pix
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Epoch 1/20
249/250 [============================>.] - ETA: 0s - loss: 4.1364 - acc: 0.2600Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
250/250 [==============================] - 42s 169ms/step - loss: 4.1305 - acc: 0.2605 - val_loss: 2.6649 - val_acc: 0.3605
Epoch 2/20
250/250 [==============================] - 37s 147ms/step - loss: 1.3803 - acc: 0.6194 - val_loss: 1.1007 - val_acc: 0.7706
Epoch 3/20
250/250 [==============================] - 37s 146ms/step - loss: 0.9906 - acc: 0.7967 - val_loss: 0.8961 - val_acc: 0.8289
Epoch 4/20
250/250 [==============================] - 37s 147ms/step - loss: 0.8399 - acc: 0

Epoch 15/20
250/250 [==============================] - 36s 145ms/step - loss: 0.5343 - acc: 0.8873 - val_loss: 0.5203 - val_acc: 0.8974
Epoch 16/20
250/250 [==============================] - 36s 143ms/step - loss: 0.5334 - acc: 0.8873 - val_loss: 0.4996 - val_acc: 0.8992
Epoch 17/20
250/250 [==============================] - 36s 143ms/step - loss: 0.5315 - acc: 0.8876 - val_loss: 0.5021 - val_acc: 0.9007
Epoch 18/20
100/100 [==============================] - 3s 34ms/step - loss: 0.5153 - acc: 0.8982

Epoch 00018: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-07.
250/250 [==============================] - 33s 134ms/step - loss: 0.5297 - acc: 0.8879 - val_loss: 0.5153 - val_acc: 0.8982
Epoch 19/20
100/100 [==============================] - 3s 34ms/step - loss: 0.4975 - acc: 0.9005

Epoch 00019: ReduceLROnPlateau reducing learning rate to 9.999999974752428e-08.
250/250 [==============================] - 33s 134ms/step - loss: 0.5283 - acc: 0.8882 - val_loss: 0.4975 - val_

# Experiment C – Fast test

Train all models under test for 50 training steps. Repeat each experiment for 20 `iterations`.




In [6]:
# Results csv saved with this filename
test_title = '256px_50steps'

# Experiment configuration
training_params = dict(
    train_steps=50, 
    val_steps=100, 
    epochs=1, 
    iterations=20, 
    lr=1e-4
)

# Train models and record results
for model in models:
    print(f'\nTESTING MODEL: {model[0]}')
    results = test_model(model[1], train_loader, test_loader, **training_params, 
                         model_params=model[2])
    results_df = hists2df(results)
    results_df.to_csv(f'{results_pth}{model[0]}_{test_title}.csv')


TESTING MODEL: unet_pix2pix
50/50 [==============================] - 13s 250ms/step - loss: 9.2893 - acc: 0.2209 - val_loss: 5.7268 - val_acc: 0.2031


## Parameter count

Count parameters in model

In [5]:
# Count trainable parameters for each model
params = []
for model in models:
    m = model[1](**model[2])
    params.append([model[0], m.count_params()])

reinitializing layer block1_conv1.kernel
reinitializing layer block1_conv1.bias
reinitializing layer block1_conv2.kernel
reinitializing layer block1_conv2.bias
reinitializing layer block2_conv1.kernel
reinitializing layer block2_conv1.bias
reinitializing layer block2_conv2.kernel
reinitializing layer block2_conv2.bias
reinitializing layer block3_conv1.kernel
reinitializing layer block3_conv1.bias
reinitializing layer block3_conv2.kernel
reinitializing layer block3_conv2.bias
reinitializing layer block3_conv3.kernel
reinitializing layer block3_conv3.bias
reinitializing layer block4_conv1.kernel
reinitializing layer block4_conv1.bias
reinitializing layer block4_conv2.kernel
reinitializing layer block4_conv2.bias
reinitializing layer block4_conv3.kernel
reinitializing layer block4_conv3.bias
reinitializing layer block5_conv1.kernel
reinitializing layer block5_conv1.bias
reinitializing layer block5_conv2.kernel
reinitializing layer block5_conv2.bias
reinitializing layer block5_conv3.kernel

In [8]:
pd.DataFrame(params, columns=['model', 'parameters'])

,model,parameters
0,unet_baseline,31031685
1,unet_baseline_upsampled,31031685
2,unet_ternaus_nopre,28470791
3,unet_ternaus,28467719
4,unet_ternaus_drop,28470791
5,unet_ternaus_bn,28495879
6,unet_ternaus_dropbn,28495879
7,unet_pix2pix,54417665
